<a href="https://colab.research.google.com/github/LuFernandez/PASA/blob/master/TP2.%2520Laboratorio%2520de%2520filtrado%2520%C3%B3ptimo/informe2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filtrado óptimo

Procesamiento Adaptativo de Señales Aleatorias

© 2019 Parra, Rocío

© 2019 Fernandez, Lucero Guadalupe

In [0]:
from IPython.display import clear_output
from scipy.io.wavfile import read
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import wave, struct, math, random
from IPython.display import Audio
from scipy import signal as sgn
from scipy.linalg import toeplitz
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import ccf

In [0]:
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/samples/sofunny.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/samples/lorde.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/samples/sweeplin.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/samples/sweeplog.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/samples/ruido.wav"

!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/records/grab-sofunny.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/records/grab-lorde.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/records/grab-sweeplin.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/records/grab-sweeplog.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2.%20Laboratorio%20de%20filtrado%20óptimo/records/grab-ruido.wav"
clear_output()

In [0]:

def get_mono_array(stereo_file_name):
  file = read(stereo_file_name)
  fs = file[0]
  datos = np.array(file[1],dtype=float) 
  if len(np.shape(datos))>1: #si tiene dos canales
      ch0 = datos[:, 0]
      ch1 = datos[:, 1]
      datos = (ch0 + ch1)/2  # o lo hacemos mono
  return fs, datos
  
def get_signal(path):
  file_name = path
  fs, grab = get_mono_array(file_name)
  grab =  grab/np.linalg.norm(grab)
  return grab


In [23]:
signals = {
  "music" : {  "u" : get_signal('./lorde.wav'),  "d" : get_signal('./grab-lorde.wav'),  "shift" : 1  },
  "voice" : {  "u" : get_signal('./sofunny.wav'),  "d" : get_signal('./grab-sofunny.wav'),  "shift" : 1  },
  "lin" : {  "u" : get_signal('./sweeplin.wav'),  "d" : get_signal('./grab-sweeplin.wav'),  "shift" : 1  },
  "log" : {  "u" : get_signal('./sweeplog.wav'),  "d" : get_signal('./grab-sweeplog.wav'),  "shift" : 1  },
  "noise" : {  "u" : get_signal('./ruido.wav'),  "d" : get_signal('./grab-ruido.wav'),  "shift" : 1  }
}

/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [0]:
#ordenes para el filtro
Ms = [i for i in range(1,11)]
Ms = np.concatenate((Ms,[10*i for i in range(2,11)]))
Ms = np.concatenate((Ms,[100*i for i in range(2,11)]))
Ms = np.concatenate((Ms,[1000*i for i in range(2,7)]))